In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact
from scipy.signal import freqz
%matplotlib inline

from ipywidgets import interact, Dropdown, IntSlider
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display


In [115]:
audio_data = pd.read_csv("audio_features.csv")
metadata_df = pd.read_csv("metadata.csv")
all_data = pd.read_csv("all_features.csv")

In [116]:
display(all_data.head())
display(metadata_df.head())

,smcAC_case001,smcDC_case001,vib_table_case001,vib_spindle_case001,AE_table_case001,AE_spindle_case001,smcAC_case002,smcDC_case002,vib_table_case002,vib_spindle_case002,...,AE_table_case164,AE_spindle_case164,smcAC_case165,smcDC_case165,vib_table_case165,vib_spindle_case165,AE_table_case165,AE_spindle_case165,smcAC_case166,smcDC_case166
0,-0.017090,0.625000,0.078125,0.314941,0.087280,0.103760,0.307617,0.668945,0.075684,0.301514,...,0.114746,0.139771,0.244141,1.328125,0.063477,0.290527,0.101318,0.117798,-0.205078,1.381836
1,0.263672,0.810547,0.085449,0.301514,0.098267,0.123291,0.312500,0.678711,0.080566,0.308838,...,0.095215,0.112305,0.244141,1.333008,0.058594,0.279541,0.121460,0.140381,-0.239258,1.386719
2,0.207520,0.781250,0.078125,0.303955,0.092163,0.104980,0.195312,0.859375,0.078125,0.299072,...,0.101929,0.120239,0.205078,1.333008,0.058594,0.285645,0.122070,0.142822,-0.156250,1.386719
3,0.302734,0.849609,0.073242,0.300293,0.095215,0.111084,0.187988,1.123047,0.080566,0.306396,...,0.098877,0.118408,0.156250,1.333008,0.063477,0.286865,0.119019,0.139160,0.126953,1.381836
4,0.239258,1.098633,0.083008,0.299072,0.083008,0.092163,-0.009766,1.166992,0.075684,0.319824,...,0.098877,0.120239,0.048828,1.337891,0.058594,0.286865,0.114136,0.132446,0.200195,1.376953


,Unnamed: 0,case,run,VB,time,DOC,feed,material
0,NaN,1.0,1.0,0.0,2.0,1.5,0.5,1.0
1,NaN,1.0,2.0,NaN,4.0,1.5,0.5,1.0
2,NaN,1.0,3.0,NaN,6.0,1.5,0.5,1.0
3,NaN,1.0,4.0,11.0,7.0,1.5,0.5,1.0
4,NaN,1.0,5.0,NaN,11.0,1.5,0.5,1.0


In [117]:
signal = audio_data["AE_spindle_case001"] # Choose a signal to analyze

# Analyze STFT of Each Label

In [118]:

bins = [0, 20, 40, 70, 100, 150, 1_000]
labels = ["Good", "Slight", "Average", "Heavy", "Severe", "Failure"]

labels = pd.cut(metadata_df["VB"], bins=bins, labels=labels)
# Remove case 166 and 167 since no data for them

label_dict = {"Good": [], "Slight": [], "Average": [], "Heavy": [], "Severe": [], "Failure": []}

for label in label_dict:
    label_dict[label] = labels[labels==label].index + 1
    

Important Notes: <br>
* There are 2 different speeds.
* There are 2 different cutting materials.

# Filter Bank Analysis

In [119]:
from scipy.signal import butter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = fs / 2
    low = lowcut / nyq
    high = highcut / nyq
    
    return butter(order, [low, high], btype='band')


In [120]:
def apply_filter_bank(sensor_type, case_number):
    fs = 250
    case_str = str(case_number).zfill(3)

    signal = audio_data[f"{sensor_type}_case{case_str}"]
    
    bands = {
        "band 1": (0.01, 20),    
        "band 2": (20, 60),  
        "band 3": (60, 100),
        "band 4": (100, 124.99) 
    }

    band_energies = {}
    total_energy = np.sum(signal ** 2)

    # Why are we cycling through each band x2?
    for band, (low, high) in bands.items():
        b, a = butter_bandpass(low, high, fs)
        filtered = filtfilt(b, a, signal)
        energy = np.sum(filtered ** 2)
        band_energies[band] = energy / total_energy if total_energy > 0 else 0
        print("Band", band, "Energies", band_energies[band])
    
    print("Band items", bands.items())

    # Access band_energy_ratios['low'], ['mid'], ['high']

    print(band_energies)

    plt.bar(band_energies.keys(), band_energies.values())
    plt.title(f"Band Energy Ratios - Case {case_str}")
    plt.ylabel("Energy Ratio")
    plt.show()



def create_category_plotter(category_name, label_dict):
    valid_cases = sorted(label_dict[category_name])
    sensor_options = ["AE_spindle", "AE_table", "vib_spindle", "vib_table", "smcAC", "smcDC"]
    
    interact(
        apply_filter_bank,
        sensor_type=Dropdown(options=sensor_options, description="Sensor"),
        case_number=Dropdown(options=valid_cases, description="Case #"),
        category=widgets.fixed(category_name)
    )

    

# Create one interactive plotter per category
for category in label_dict.keys():
    print(f"\n⏬ {category} category visualization")
    create_category_plotter(category, label_dict)


⏬ Good category visualization


interactive(children=(Dropdown(description='Sensor', options=('AE_spindle', 'AE_table', 'vib_spindle', 'vib_ta…


⏬ Slight category visualization


interactive(children=(Dropdown(description='Sensor', options=('AE_spindle', 'AE_table', 'vib_spindle', 'vib_ta…


⏬ Average category visualization


...

interactive(children=(Dropdown(description='Sensor', options=('AE_spindle', 'AE_table', 'vib_spindle', 'vib_ta…


⏬ Severe category visualization


interactive(children=(Dropdown(description='Sensor', options=('AE_spindle', 'AE_table', 'vib_spindle', 'vib_ta…


⏬ Failure category visualization


interactive(children=(Dropdown(description='Sensor', options=('AE_spindle', 'AE_table', 'vib_spindle', 'vib_ta…

# Brief explanation of the energy conversion process. 
* How did we calculate energy? 
* Why use energy as a metric?